# The Billionaire Problem

In recent years, billionaires have gone from objects of admiration to outright villians in certain political and social circles. The (over?)simplified idea is that we have "a billionaire problem" in our society, i.e. they are a sign of massive income/wealth inequality and a failure of capitalism to provide general prosperity. Is this true? This notebook tries to interrogate that question.